# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,38.23,81,14,9.37,GS,1698703487
1,1,pimenta bueno,-11.6725,-61.1936,83.14,62,71,6.42,BR,1698703488
2,2,margaret river,-33.9500,115.0667,48.52,90,47,8.84,AU,1698703488
3,3,kuanda,56.3161,116.0806,9.19,55,37,0.65,RU,1698703489
4,4,puerto natales,-51.7236,-72.4875,44.08,51,64,3.78,CL,1698703489


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    size = "Humidity",
    scale = 0.85,
    alpha = 0.5,
    color = "City"   

)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

Criteria is: Humidity-- has humidity higher than 0

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
Humidata = city_data_df.loc[(city_data_df["Humidity"] != 0)]

# Drop any rows with null values
Humidata.drop([8], axis=0, inplace=True)

# Display sample data
Humidata.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,38.23,81,14,9.37,GS,1698703487
1,1,pimenta bueno,-11.6725,-61.1936,83.14,62,71,6.42,BR,1698703488
2,2,margaret river,-33.9500,115.0667,48.52,90,47,8.84,AU,1698703488
3,3,kuanda,56.3161,116.0806,9.19,55,37,0.65,RU,1698703489
4,4,puerto natales,-51.7236,-72.4875,44.08,51,64,3.78,CL,1698703489


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,grytviken,-54.2811,-36.5092,38.23,81,14,9.37,GS,1698703487,
1,1,pimenta bueno,-11.6725,-61.1936,83.14,62,71,6.42,BR,1698703488,
2,2,margaret river,-33.9500,115.0667,48.52,90,47,8.84,AU,1698703488,
3,3,kuanda,56.3161,116.0806,9.19,55,37,0.65,RU,1698703489,
4,4,puerto natales,-51.7236,-72.4875,44.08,51,64,3.78,CL,1698703489,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# get latitude, longitude from the DataFrame
lng = row["Lng"]
lat = row["Lat"]
categories = "accommodation.hotel"
#categories = f"accommodation.{hotel}"
radius = 10000

# Add filter and bias parameters with the current city's latitude and longitude
# to the params dictionary
filter = f"circle:{lng},{lat},{radius}"
bias = f"proximity:{lng},{lat},{radius}"
# no limit

# Set parameters to search for a hotel
params = {
    "categories":categories,
    "radius": 10000,
    "filter":filter,
    "bias":bias,
    "key": geoapify_key}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [54]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
pimenta bueno - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
kuanda - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
mount pearl - nearest hotel: No hotel found
tazovsky - nearest hotel: No hotel found
ipora - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
lethbridge - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
peterhead - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
college - nearest hotel: No hotel found
manismata - nearest hotel: No hotel found
northam - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
dul'durga - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,grytviken,-54.2811,-36.5092,38.23,81,14,9.37,GS,1698703487,No hotel found
1,1,pimenta bueno,-11.6725,-61.1936,83.14,62,71,6.42,BR,1698703488,No hotel found
2,2,margaret river,-33.9500,115.0667,48.52,90,47,8.84,AU,1698703488,No hotel found
3,3,kuanda,56.3161,116.0806,9.19,55,37,0.65,RU,1698703489,No hotel found
4,4,puerto natales,-51.7236,-72.4875,44.08,51,64,3.78,CL,1698703489,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
575,575,borogontsy,62.6683,131.1683,8.78,93,100,6.87,RU,1698703731,No hotel found
576,576,arraial do cabo,-22.9661,-42.0278,76.86,88,0,4.61,BR,1698703823,No hotel found
577,577,laguna,38.4210,-121.4238,76.08,27,0,5.75,US,1698703605,No hotel found
578,578,shanga,11.2106,4.5788,83.25,30,77,1.45,NG,1698703824,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
%%capture --no-display
# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    color = "Hotel"   
)

# Display the map plot
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]